## 부정청약, 위장전입, 불법매매에 해당할 확률의 순위를 기반으로 조사대상을 반환하는 코드

### 1-1. 환경설정 및 데이터 로딩

In [45]:
# 라이브러리 로드 및 환경설정 (아래 라이브러리가 모두 설치되어야함)
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.ensemble  import RandomForestClassifier
from sklearn import metrics 
import xgboost as xgb   
from xgboost import XGBClassifier
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

import warnings
import warnings
warnings.filterwarnings(action='ignore')

# 자료 읽기 
DF=pd.read_csv(r'final_add.csv', encoding='CP949', sep=",")
DF_test=pd.read_csv(r'test2_add.csv', encoding='CP949', sep=",")

#DF_add = DF[DF['부정청약판정'] == 0].sample(n=400, random_state=1004)

# 학습 세트의 경우, 실제 조사를 한 경우만 로딩
cond=DF['검사여부']==1
DF_train=DF[cond]
#DF_test = pd.concat([DF_test, DF_add])

### 1-2. 위험순위 및 사전설정 함수 정의

In [46]:
# 위험순위에 대한 quantile points 설정 (1 기준. 예시: 0.5 = 상위 50% 추출)

measurements=[0.5, 0.25, 0.1]
#measurements=[0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1]


# 확률과 quantile point를 입력값으로 조사 결정을 변환하는 함수

def convert(pred_prob,m): 
    e=[]
    for h in range(0,len(pred_prob)):
        e.append(pred_prob[h][1])
    e_temp=sorted(e)
    e_temp.reverse()
    e_ind=[]
    for i in e:
        e_ind.append(e_temp.index(i)+1)    
    last_ind=len(e_ind)
    init=0
    for x in range(0,last_ind):
        dd=e[x]
        if dd == 0:
            e_ind[x]=last_ind
    point_m=int(round(len(e_ind)*m))
    pred=[]
    
    for x in range(0,last_ind):
        hh=e_ind[x]
    
        if hh<=point_m:
            pred.append(1)
        else:
            pred.append(0)

    return pred

 # 조사 시 검거 확률과 미스 확률을 반환하는 함수 (성능 테스트용)

def performance(X, Y):
    detect=[]
    miss=[]
    last_ind=len(Y)
    for i in range(0,last_ind):
        x_i=X[i]
        y_i=Y[i]

    
        if (x_i==1) & (y_i==1):
            detect.append(1)
        elif (x_i==1) & (y_i==0):
            detect.append(0)
        elif (x_i==0) & (y_i==1):
            miss.append(1)
        elif (x_i==0) & (y_i==0):
            miss.append(0)    
    prob_detect=detect.count(1)/len(detect)
    prob_miss=miss.count(1)/len(miss)       
    return [prob_detect,prob_miss]

# 자료 인코딩용 함수 정의 : 아래 함수에서 dataDF는 데이터프레임, list는 라벨화하려는 범주변수의 목록
def encode_labels(list, dataDF):                               
    for x in list:
        temp=preprocessing.LabelEncoder()
        dataDF[x]=temp.fit_transform(dataDF[x])
    return dataDF

### 1-3. 시행할 내용의 설정 (시행 시=1, 시행 안할 시=0)

In [47]:
#------------- 각 flag는 어떤 test를 할 것인지 설정.  RF는 random forest, XGB는 XGBoost, Y1,Y2,Y3는 부정청약,위장전입,부정매매 test
flag_RF=1
flag_XGB=1

flag_Y1=1
flag_Y2=1
flag_Y3=1

### 2-1. random forest 및 XGBoost의 하이퍼 파라메터 설정

In [48]:
#------- RF parameters----------

max_depth_Y1=10
max_depth_Y2=7
max_depth_Y3=14

min_samples_leaf_Y1=5
min_samples_leaf_Y2=3
min_samples_leaf_Y3=3

max_leaf_nodes_Y1=None
max_leaf_nodes_Y2=None
max_leaf_nodes_Y3=None

max_features_Y1=10
max_features_Y2=10
max_features_Y3=6

min_samples_split_Y1=4
min_samples_split_Y2=4
min_samples_split_Y3=4

bootstrap_Y1=True
bootstrap_Y2=True
bootstrap_Y3=True

warm_start_Y1=False
warm_start_Y2=False
warm_start_Y3=False

class_weight_Y1={0:1, 1:1}
class_weight_Y2={0:1, 1:1}
class_weight_Y3={0:1, 1:1}

#-------XGBoost parameters-------

reg_alpha_Y1 = 0.75
reg_alpha_Y2 = 0.75
reg_alpha_Y3 = 0.75

reg_lambda_Y1 = 0.5
reg_lambda_Y2 = 0.5
reg_lambda_Y3 = 0.5

gamma_Y1 = 0
gamma_Y2 = 0
gamma_Y3 = 0

booster_Y1 = 'gbtree'
booster_Y2 = 'gbtree'
booster_Y3 = 'gbtree'
#  'gbtree', 'gblinear' 'dart'

max_depth_Y1 = 10
max_depth_Y2 = 10
max_depth_Y3 = 10 

objective_Y1 = 'binary:logistic'
objective_Y2 = 'binary:logistic'
objective_Y3 = 'binary:logistic'
#'binary:logistic', 'binary:logitraw', 'binary:hinge'

learning_rate_Y1=0.75
learning_rate_Y2=0.75
learning_rate_Y3=0.75

min_child_weight_Y1=1
min_child_weight_Y2=1
min_child_weight_Y3=1

colsample_bytree_Y1=1
colsample_bytree_Y2=1
colsample_bytree_Y3=1

scale_pos_weight_Y1=1
scale_pos_weight_Y2=1
scale_pos_weight_Y3=1

subsample_Y1=1
subsample_Y2=1
subsample_Y3=1

In [49]:
DF_train.describe()

,Unnamed: 0,주택관리번호,동,호수,공급유형,특별공급종류,공급위치_시군구코드,크기,일반공급,특별공급,공급금액,생년,연령,변경일자,주소일치여부_부동산원,주소중복횟수,특일동시여부,2년청약건수,세대원수,분리세대원수,폰중복횟수_부동산원,IP중복당첨횟수_부동산원,IP중복신청횟수_부동산원,폰번호,ad_IP중복_2자리,ad_IP중복_3자리,ad_IP중복_4자리,접수일자,가점신청여부,가점당첨여부,가점합계,ad_신혼미공특소득구분,청약_재당첨제한여부,청약_혼인및한부모가족,세대주여부,과거5년이내당첨,무주택세대구성원,소득기준,소득(소득세 5개년도 납부)기준,세대구성,무주택세대주,자산기준,노부모부양,3명이상미성년자녀,혼인기간7년이내,ad_주소일치여부,검사여부,부정청약판정,부정_거주지유지,부정_주소지유지,부정_위장전입,부정_자격위조,부정_입주자저축증서매매,부정_위장이혼,부정_기타,모집공고일,당첨자발표일,해당전입제한일,1순위요건,총공급가구수,분양가상한제여부,과밀성장권역여부,1순위인터넷접수일,2순위인터넷접수일시,ad_변경일자,ad_총공급,ad_성명생년중복,ad_성명생년전화중복,ad_행정변경시점,ad_접수시간,ad_신청당첨거주일치여부,ad_부양가족수,ad_저축가입기간,ad_무주택기간,ad_청약납부회차,ad_청약납부금액,ad_청약경과기간,ad_다자녀영유아자녀수,ad_다자녀무주택기간,ad_다자녀해당지역거주기간,ad_다자녀입주자저축가입기간,ad_다자녀총점,ad_다자녀미성년자녀수,ad_신혼미공특미성년자녀수,ad_신혼미공특태아수,ad_미성년자녀수,ad_총점,ad_변경시점2
count,12968.000000,1.296800e+04,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,1.296800e+04,12968.00000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,1.296800e+04,12968.000000,12968.000000,12968.000000,1.296800e+04,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.0,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,1.296800e+04,1.296800e+04,1.296800e+04,12968.000000,12968.000000,12968.000000,12968.000000,1.296800e+04,1.296800e+04,1.296800e+04,12968.000000,12968.000000,12968.0,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,1.296800e+04,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000,12968.000000
mean,37481.197795,2.021038e+09,255.376465,1307.869679,15.285318,1.542181,37371.448334,81.355666,90.880167,96.960750,50771.642736,767242.413325,43.538402,2.013558e+07,0.65145,2.761721,0.291641,0.261104,3.063772,0.444556,0.476789,0.469540,3.037014,1.058699e+09,427.388495,40.281616,0.176743,2.021179e+07,0.421499,0.306601,21.545188,0.013495,0.032927,0.026758,0.109577,0.010487,0.455043,0.211444,0.127622,0.088371,0.018584,0.127159,0.020435,0.074337,0.165407,0.772980,1.0,0.040176,0.000231,0.000694,0.027761,0.001774,0.008791,0.001542,0.001002,2.021168e+07,2.021217e+07,2.020300e+07,17.163479,645.193091,0.381015,0.323180,2.021180e+07,2.021180e+07,2.013638e+07,187.840916,1.000771,0.0,2763.232341,490.343846,0.756323,1.488202,4.189775,5.336444,14.240052,5.674902e+06,33.985194,0.176974,0.245450,0.252082,0.095003,5.114513,0.095312,0.353486,0.202036,0.677591,5.377853,2751.936922
std,19728.452483,6.614965e+05,368.880649,869.115390,4.992045,1.767790,9368.485725,15.221627,82.758740,89.083979,20585.643521,135255.302627,12.652874,9.318913e+04,0.47653,9.620250,0.459765,0.967104,1.697040,0.936898,0.639984,1.021628,21.352918,2.897264e+07,1216.955040,143.873156,0.880581,6.496941e+03,0.493818,0.461100,27.474278,0.130443,0.178453,0.161382,0.312374,0.101873,0.497994,0.408348,0.333681,0.283845,0.135056,0.333164,0.141488,0.262328,0.371562,0.418922,0.0,0.196379,0.015209,0.026336,0.164293,0.042078,0.093350,0.039243,0.031647,6.556619e+03,6.156097e+03,8.384712e+03,8.636072,409.406184,0.485655,0.467709,6.496825e+03,6.497133e+03,9.263291e+04,154.929174,0.027760,0.0,3379.360743,213.212071,0.429316,1.994250,5.733510,7.416126,40.223609,5.596372e+06,51.846028,0.687495,0.899763,0.921327,0.351374,18.078654,0.362101,0.861489,0.483447,1.344065,18.076141,3379.319546
min,2.000000,2.020001e+09,1.000000,101.000000,10.000000,0.000000,11305.000000,30.677400,1.000000,0.000000,14840.000000,103.000000,18.000000,1.968082e+07,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.039923e+08,0.000000,0.000000,0.000000,2.020063e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000

In [50]:
print(DF_test.columns)
DF_test.describe()

Index(['Unnamed: 0', '주택관리번호', '주택명', '동', '호수', '공급유형', '특별공급종류',
       '공급위치_시군구코드', '크기', '일반공급',
       ...
       'ad_다자녀무주택기간', 'ad_다자녀해당지역거주기간', 'ad_다자녀입주자저축가입기간', 'ad_다자녀총점',
       'ad_다자녀미성년자녀수', 'ad_신혼미공특미성년자녀수', 'ad_신혼미공특태아수', 'ad_미성년자녀수', 'ad_총점',
       'ad_변경시점2'],
      dtype='object', length=146)


,Unnamed: 0,주택관리번호,동,호수,공급유형,특별공급종류,공급위치_시군구코드,크기,일반공급,특별공급,공급금액,생년,연령,변경일자,배우자,주소일치여부_부동산원,주소중복횟수,특일동시여부,2년청약건수,세대원수,분리세대원수,폰중복횟수_부동산원,IP중복당첨횟수_부동산원,IP중복신청횟수_부동산원,폰번호,IP_1,IP_2,IP_3,IP_4,ad_IP중복_2자리,ad_IP중복_3자리,ad_IP중복_4자리,접수일자,가점신청여부,가점당첨여부,가점합계,ad_신혼미공특소득구분,신혼공특_월평균소득배점,신혼공특_혼인기간,청약_재당첨제한여부,청약_혼인및한부모가족,세대주여부,과거5년이내당첨,무주택세대구성원,소득기준,소득(소득세 5개년도 납부)기준,세대구성,무주택세대주,자산기준,노부모부양,3명이상미성년자녀,혼인기간7년이내,ad_주소일치여부,검사여부,부정청약판정,부정_거주지유지,부정_주소지유지,부정_위장전입,부정_자격위조,부정_입주자저축증서매매,부정_위장이혼,부정_기타,모집공고일,당첨자발표일,해당전입제한일,1순위요건,총공급가구수,분양가상한제여부,과밀성장권역여부,1순위인터넷접수일,2순위인터넷접수일시,ad_변경일자,ad_총공급,ad_성명생년중복,ad_성명생년전화중복,ad_행정변경시점,ad_접수시간,ad_신청당첨거주일치여부,ad_부양가족수,ad_저축가입기간,ad_무주택기간,ad_청약납부회차,ad_청약납부금액,ad_청약경과기간,ad_다자녀영유아자녀수,ad_다자녀무주택기간,ad_다자녀해당지역거주기간,ad_다자녀입주자저축가입기간,ad_다자녀총점,ad_다자녀미성년자녀수,ad_신혼미공특미성년자녀수,ad_신혼미공특태아수,ad_미성년자녀수,ad_총점,ad_변경시점2
count,16491.00000,1.649100e+04,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,1.649100e+04,16491.000000,16491.000000,16491.0,16491.000000,16491.0,16491.000000,16490.000000,16491.000000,16491.000000,16491.000000,1.649100e+04,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,1.649100e+04,16491.000000,16491.000000,16491.000000,16491.0,16491.0,16491.0,16491.0,16491.0,16491.000000,16491.0,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.0,16491.0,16491.0,16491.0,16491.0,16491.0,16491.0,16491.0,16491.0,1.649100e+04,1.649100e+04,1.649100e+04,16491.000000,16491.000000,16491.000000,16491.000000,1.649100e+04,1.649100e+04,1.649100e+04,16491.000000,16491.000000,16491.0,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,1.649100e+04,16491.000000,16491.000000,16491.000000,16491.00000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000,16491.000000
mean,8245.00000,2.022043e+09,717.184040,1134.307562,14.033109,1.087199,36847.527015,83.205106,96.274877,134.106422,54627.412771,799966.509611,41.474562,2.013319e+07,0.152204,0.725790,0.0,0.216906,0.0,2.878964,0.078169,0.235462,0.108120,1.557213,1.058582e+09,134.727670,119.442302,115.906070,115.968892,247.305561,21.471348,0.108120,2.022259e+07,0.378995,0.172640,13.512098,0.0,0.0,0.0,0.0,0.0,0.030805,0.0,0.076890,0.263416,0.139046,0.028318,0.001577,0.291735,0.012067,0.024741,0.108301,0.725790,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.022138e+07,2.022268e+07,2.021363e+07,10.483536,777.672245,0.523983,0.290219,2.022259e+07,2.022259e+07,2.013319e+07,230.381299,1.000364,0.0,3315.601237,460.758110,0.835425,1.038324,3.424353,2.685101,14.855618,6.059055e+06,29.081863,0.070463,0.088169,0.10527,0.038688,1.973198,0.033533,0.296222,0.181190,0.419623,2.492572,3303.968286
std,4760.68598,2.014824e+05,1246.000739,778.034990,4.905770,1.475422,11563.186473,15.042791,64.937105,121.896499,13727.211317,117721.793928,10.770172,9.469538e+04,0.359230,0.446129,0.0,0.412151,0.0,1.269945,0.422130,0.451579,0.727116,17.203305,2.623304e+07,74.590809,82.028633,81.673301,77.827430,558.784576,58.135175,0.727116,3.476545e+03,0.485151,0.377947,19.914256,0.0,0.0,0.0,0.0,0.0,0.172793,0.0,0.266426,0.440500,0.346005,0.165886,0.039677,0.454575,0.109189,0.155339,0.310770,0.446129,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.872276e+03,3.531155e+03,6.761488e+03,7.268348,297.149091,0.499440,0.453877,3.474618e+03,3.474614e+03,9.469538e+04,152.343594,0.019072,0.0,3457.404808,227.834805,0.370807,1.545152,5.157168,5.524200,39.910003,5.456953e+06,49.535101,0.446000,0.539962,0.61728,0.233791,11.429888,0.201338,0.743761,0.434043,0.988601,11.561442,3457.261326
min,0.00000,2.022000e+09,101.000000,101.000000,10.000000,0.000000,11260.000000,39.990000,1.000000,0.000000,28880.000000,101.000000,18.000000,1.968102e+07,0.000000,0.000000,0.0,0.000000,0.0,1.000000,0.000000,0.000000,0.000000,0.000000,1.075710e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.02

### 2-2. 변수처리

In [51]:
# 사용할 변수 리스트

X_list=['크기', 'ad_총공급', 'ad_성명생년전화중복', 'ad_행정변경시점',\
        '배우자', '2년청약건수', '세대원수', '분리세대원수', '폰중복횟수_부동산원', 'IP중복신청횟수_부동산원',\
        'ad_IP중복_3자리', 'ad_IP중복_4자리', 'ad_접수시간', 'ad_신청당첨거주일치여부', 'ad_부양가족수', 'ad_저축가입기간', 'ad_무주택기간',\
        'ad_청약납부회차', 'ad_청약경과기간', 'ad_총점', 'ad_주소일치여부', 'ad_변경시점2',\
        '공급금액', '연령', '세대주관계', '특일동시여부', '주소중복횟수', '가점합계', 'ad_신청유형', '기관추천종류',\
        '접수매체', '주택소유구분', '장기복무군인', '분양가상한제여부'
        ]
# 범주형 변수 선언
str_list = ['배우자','세대주관계', 'ad_신청유형', '기관추천종류', '접수매체', '주택소유구분', '장기복무군인']


# 결측치 제거 위해 테스트 대상 특정
#DF_train_selected=DF_train[X_list+['부정청약판정', '부정_위장전입', '부정_입주자저축증서매매']]
#DF_test_selected=DF_test[X_list]

# 결측치있는 표본 제거
DF_train_selected=DF_train.dropna()
DF_test_selected=DF_test.dropna()
print(len(DF_train)-len(DF_train_selected))
print(len(DF_test)-len(DF_test_selected))

# 설명변수 설정
X_train=DF_train_selected[X_list]
X_test=DF_test_selected[X_list]

# 예측변수 설정
Y1=DF_train_selected['부정청약판정']
Y2=DF_train_selected['부정_위장전입']
Y3=DF_train_selected['부정_입주자저축증서매매']


Y1_test=DF_test_selected['부정청약판정']
Y2_test=DF_test_selected['부정_위장전입']
Y3_test=DF_test_selected['부정_입주자저축증서매매']


# 범주형 변수 라벨화

X_train[ str_list ]=X_train[ str_list ].astype('str')
X_test[ str_list ]=X_test[ str_list ].astype('str')

tmp = pd.DataFrame()
for x in str_list:
        temp=preprocessing.LabelEncoder()
        tmp[x]=temp.fit_transform(X_test[x])
        print(x)
        mapping = dict(zip(temp.classes_, range(1, len(temp.classes_)+1)))
        print(mapping)
        

X_train=encode_labels(str_list, X_train)
X_test=encode_labels(str_list, X_test)



0
1
배우자
{'0': 1, '1': 2}
세대주관계
{'고모': 1, '누나': 2, '누이': 3, '동거인': 4, '동서': 5, '매': 6, '며느리': 7, '모': 8, '배우자': 9, '배우자의자녀': 10, '본인': 11, '부': 12, '사위': 13, '손': 14, '손부': 15, '시가친척': 16, '시누이': 17, '시모': 18, '시숙': 19, '언니': 20, '오빠': 21, '외손': 22, '외숙모': 23, '이종': 24, '자녀': 25, '장모': 26, '장인': 27, '제': 28, '조모': 29, '조카': 30, '처가친척': 31, '처남': 32, '처남댁': 33, '처제': 34, '처조카': 35, '처형': 36, '친척': 37, '형': 38}
ad_신청유형
{'기관추천': 1, '다자녀': 2, '무': 3, '신혼공특미적용': 4}
기관추천종류
{'10년이상복무군인': 1, '국가유공자 등(35조, 36조)': 2, '국가유공자 등(45조)': 3, '귀국공무원 등': 4, '납북피해자': 5, '다문화가족': 6, '무': 7, '북한이탈주민': 8, '시책추진대상(투자촉진,전통문화)': 9, '의상자,의사자유족': 10, '장기복무제대군인': 11, '장애인': 12, '중소기업근로자': 13, '철거주택(35조,36조)': 14}
접수매체
{'모바일앱': 1, '사업주체(견본주택)': 2, '사업주체(추첨대상)': 3, '인터넷': 4, '창구': 5}
주택소유구분
{'-': 1, '1주택(처분서약)': 2, '1주택이상(처분미서약)': 3, '1주택자': 4, '2주택이상자': 5, '무주택자': 6}
장기복무군인
{'N': 1, 'T': 2, 'Y': 3}


### 3. 기계학습예측기의 학습

In [52]:
if flag_RF==1:
    if flag_Y1==1:
        rf1=RandomForestClassifier(n_estimators=100, random_state=11, max_depth=max_depth_Y1, min_samples_leaf=min_samples_leaf_Y1,\
                                   max_features=max_features_Y1, max_leaf_nodes=max_leaf_nodes_Y1, min_samples_split=min_samples_split_Y1,\
                                  bootstrap=bootstrap_Y1, warm_start=warm_start_Y1, class_weight=class_weight_Y1)
        rf1.fit(X_train,Y1)
        rf1_pred = rf1.predict_proba(X_test)
        #rf1_prd = rf1.predict(X_test)
        threshold = 0.1
        rf1_prd = (rf1.predict_proba(X_test)[:, 1] > threshold).astype('float')
    if flag_Y2==1:
        rf2=RandomForestClassifier(n_estimators=100, random_state=11, max_depth=max_depth_Y2, min_samples_leaf=min_samples_leaf_Y2,\
                                   max_features=max_features_Y2, max_leaf_nodes=max_leaf_nodes_Y2, min_samples_split=min_samples_split_Y2,\
                                   bootstrap=bootstrap_Y2, warm_start=warm_start_Y2, class_weight=class_weight_Y1)
        rf2.fit(X_train,Y2)
        rf2_pred=rf2.predict_proba(X_test)
        #rf2_prd = rf2.predict(X_test)
        rf2_prd = (rf2.predict_proba(X_test)[:, 1] > threshold).astype('float')
    if flag_Y3==1:
        rf3=RandomForestClassifier(n_estimators=100, random_state=11, max_depth=max_depth_Y3, min_samples_leaf=min_samples_leaf_Y3,\
                                   max_features=max_features_Y3, max_leaf_nodes=max_leaf_nodes_Y3, min_samples_split=min_samples_split_Y3,\
                                   bootstrap=bootstrap_Y3, warm_start=warm_start_Y3, class_weight=class_weight_Y1)
        rf3.fit(X_train,Y3)
        rf3_pred=rf3.predict_proba(X_test)
        #rf3_prd = rf3.predict(X_test)
        rf3_prd = (rf3.predict_proba(X_test)[:, 1] > threshold).astype('float')
            
#-------------------------------XGB: training-----------------------------------
if flag_XGB==1:
    if flag_Y1==1:
        xgb1=XGBClassifier(reg_alpha=reg_alpha_Y1, reg_lambda=reg_lambda_Y1, gamma=gamma_Y1, booster=booster_Y1, max_depth=max_depth_Y1,\
                           objective=objective_Y1, learning_rate=learning_rate_Y1, min_child_weight=min_child_weight_Y1,\
                           colsample_bytree=colsample_bytree_Y1, scale_pos_weight=scale_pos_weight_Y1, subsample_=subsample_Y1,\
                           verbosity = 0)
        xgb1.fit(X_train,Y1)
        xgb1_pred=xgb1.predict_proba(X_test)
        xgb1_prd = xgb1.predict(X_test)
    if flag_Y2==1:
        xgb2=XGBClassifier(reg_alpha=reg_alpha_Y2,reg_lambda=reg_lambda_Y2, gamma=gamma_Y2, booster=booster_Y2, max_depth=max_depth_Y2,\
                           objective=objective_Y2, learning_rate=learning_rate_Y2, min_child_weight=min_child_weight_Y2,\
                           colsample_bytree=colsample_bytree_Y2, scale_pos_weight=scale_pos_weight_Y2, subsample_=subsample_Y2,\
                           verbosity = 0)
        xgb2.fit(X_train,Y2)
        xgb2_pred=xgb2.predict_proba(X_test)
        xgb2_prd = xgb2.predict(X_test)
    if flag_Y3==1:
        xgb3=XGBClassifier(reg_alpha=reg_alpha_Y3,reg_lambda=reg_lambda_Y3, gamma=gamma_Y3, booster=booster_Y3, max_depth=max_depth_Y3,\
                           objective=objective_Y3, learning_rate=learning_rate_Y3, min_child_weight=min_child_weight_Y3,\
                           colsample_bytree=colsample_bytree_Y3, scale_pos_weight=scale_pos_weight_Y3, subsample_=subsample_Y3,\
                           verbosity = 0)
        xgb3.fit(X_train,Y3)
        xgb3_pred=xgb3.predict_proba(X_test)
        xgb3_prd = xgb3.predict(X_test)

### 4. 결과 반환

In [53]:
    
if flag_Y1==1:
    
    if flag_RF==1:
        DF_test_selected['부정청약_rf_prob']=rf1_pred[:,1]
        for m in measurements:
            DF_test_selected['부정청약_rf_'+str(m)]=convert(rf1_pred,m)
            print('RF-부정청약여부/quantile=',m)
                
    if flag_XGB==1:
        DF_test_selected['부정청약_xgb_prob']=xgb1_pred[:,1]
        for m in measurements:
            DF_test_selected['부정청약_xgb_'+str(m)]=convert(xgb1_pred,m)
            print('XGB-부정청약여부/quantile=',m)
        
if flag_Y2==1:
    print('-------------------------------------위장전입여부----------------------------------')
    
    if flag_RF==1:
        DF_test_selected['위장전입_rf_prob']=rf2_pred[:,1]
        for m in measurements:
            DF_test_selected['위장전입_rf_'+str(m)]=convert(rf2_pred,m)
            print('RF-위장전입/quantile=',m)
                
    if flag_XGB==1:
        DF_test_selected['위장전입_xgb_prob']=xgb2_pred[:,1]
        for m in measurements:
            DF_test_selected['위장전입_xgb_'+str(m)]=convert(xgb2_pred,m)
            print('XGB-위장전입/quantile=',m)

    
if flag_Y3==1:
    print('-------------------------------------매매여부-------------------------------------')    
    
    if flag_RF==1:
        DF_test_selected['매매_rf_prob']=rf3_pred[:,1]
        for m in measurements:
            DF_test_selected['매매_rf_'+str(m)]=convert(rf3_pred,m)
            print('RF-불법매매/quantile=',m)
                
    if flag_XGB==1:
        DF_test_selected['매매_xgb_prob']=xgb3_pred[:,1]
        for m in measurements:
            DF_test_selected['매매_xgb_'+str(m)]=convert(xgb3_pred,m)
            print('XGB-불법매매/quantile=',m)

DF_test_selected.to_csv('Result_test2.csv',encoding='CP949',sep=",")

RF-부정청약여부/quantile= 0.5
RF-부정청약여부/quantile= 0.25
RF-부정청약여부/quantile= 0.1
XGB-부정청약여부/quantile= 0.5
XGB-부정청약여부/quantile= 0.25
XGB-부정청약여부/quantile= 0.1
-------------------------------------위장전입여부----------------------------------
RF-위장전입/quantile= 0.5
RF-위장전입/quantile= 0.25
RF-위장전입/quantile= 0.1
XGB-위장전입/quantile= 0.5
XGB-위장전입/quantile= 0.25
XGB-위장전입/quantile= 0.1
-------------------------------------매매여부-------------------------------------
RF-불법매매/quantile= 0.5
RF-불법매매/quantile= 0.25
RF-불법매매/quantile= 0.1
XGB-불법매매/quantile= 0.5
XGB-불법매매/quantile= 0.25
XGB-불법매매/quantile= 0.1


In [54]:
DF_test_selected.groupby('주택명').agg({
    '생년' : 'count',
    '부정청약_rf_prob' : 'mean',
    '위장전입_rf_prob' : 'mean',
    '매매_rf_prob' : 'mean'
    })

,생년,부정청약_rf_prob,위장전입_rf_prob,매매_rf_prob
주택명,,,,
갑천2 트리풀시티 엘리프(국민주택),488,0.047262,0.018625,0.014084
갑천2 트리풀시티 엘리프(민영주택),474,0.044108,0.025974,0.015073
강동 헤리티지 자이,228,0.052155,0.024892,0.021907
남천자이,127,0.043742,0.029221,0.010604
덕소 강변 신일해피트리,145,0.039054,0.023759,0.016415
동탄 A106 어울림 파밀리에,466,0.036339,0.018935,0.017381
동탄 A107 숨마데시앙,614,0.040506,0.019626,0.019445
동탄 파크릭스A51-1BL,264,0.030421,0.019946,0.011960
동탄 파크릭스A51-2BL,231,0.027870,0.017981,0.012908


In [55]:
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf2.feature_importances_
})

In [56]:
feature_importance.sort_values(by=['importance'], ascending=False)

,feature,importance
7,분리세대원수,0.101512
21,ad_변경시점2,0.081261
3,ad_행정변경시점,0.078186
22,공급금액,0.069525
0,크기,0.066199
12,ad_접수시간,0.064979
1,ad_총공급,0.061091
24,세대주관계,0.056557
23,연령,0.053398
10,ad_IP중복_3자리,0.052124


In [137]:
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf1.feature_importances_
})

In [138]:
feature_importance.sort_values(by=['importance'], ascending=False)

,feature,importance
22,공급금액,0.085752
10,ad_IP중복_3자리,0.082420
11,ad_IP중복_4자리,0.073349
7,분리세대원수,0.065733
12,ad_접수시간,0.055667
3,ad_행정변경시점,0.053909
0,크기,0.053497
21,ad_변경시점2,0.053285
1,ad_총공급,0.052917
9,IP중복신청횟수_부동산원,0.048787


In [135]:
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf3.feature_importances_
})

In [136]:
feature_importance.sort_values(by=['importance'], ascending=False)

,feature,importance
11,ad_IP중복_4자리,0.178860
9,IP중복신청횟수_부동산원,0.108197
10,ad_IP중복_3자리,0.092995
22,공급금액,0.075206
8,폰중복횟수_부동산원,0.050107
12,ad_접수시간,0.047029
23,연령,0.045859
0,크기,0.042899
1,ad_총공급,0.041933
21,ad_변경시점2,0.030900
